In [1]:
!pip install -q sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.5 MB/s eta 0:00:00


In [2]:
faq_content = """
Q: Apa itu AI?
A: AI adalah singkatan dari Artificial Intelligence, yaitu kecerdasan buatan.

Q: Apa itu machine learning?
A: Machine learning adalah cabang AI yang membuat mesin belajar dari data.

Q: Apa perbedaan supervised dan unsupervised learning?
A: Supervised learning menggunakan data berlabel, sedangkan unsupervised tidak.
"""

with open("faq.txt", "w") as f:
    f.write(faq_content.strip())

print("File faq.txt berhasil dibuat.")

File faq.txt berhasil dibuat.


In [3]:
def load_faq(filename):
    questions = []
    answers = []
    with open(filename, 'r') as f:
        lines = f.readlines()
        for i in range(0, len(lines), 3):  # per 3 baris: Q, A, newline
            q_line = lines[i].strip()
            a_line = lines[i+1].strip()
            if q_line.startswith("Q:") and a_line.startswith("A:"):
                questions.append(q_line[3:])
                answers.append(a_line[3:])
    return questions, answers

questions, answers = load_faq("faq.txt")


In [4]:
from sentence_transformers import SentenceTransformer, util
import torch

# Load model sentence transformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Encode pertanyaan FAQ
question_embeddings = model.encode(questions, convert_to_tensor=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.51k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
def chatbot(user_input):
    input_embedding = model.encode(user_input, convert_to_tensor=True)
    cos_sim = util.pytorch_cos_sim(input_embedding, question_embeddings)
    best_score, best_idx = torch.max(cos_sim, dim=1)

    if best_score.item() >= 0.6:
        return answers[best_idx]
    else:
        return "Maaf, saya tidak dapat membantu dengan pertanyaan itu."


In [6]:
print("Selamat datang di Chatbot FAQ! Ketik 'exit' untuk keluar.\n")

while True:
    user_input = input("Anda: ")
    if user_input.lower() == "exit":
        print("Chatbot: Terima kasih! Sampai jumpa.")
        break
    response = chatbot(user_input)
    print("Chatbot:", response)


Selamat datang di Chatbot FAQ! Ketik 'exit' untuk keluar.

Anda: Apa itu ai
Chatbot: AI adalah singkatan dari Artificial Intelligence, yaitu kecerdasan buatan.
Anda: Ai itu apasih
Chatbot: AI adalah singkatan dari Artificial Intelligence, yaitu kecerdasan buatan.
Anda: oke, kalau machine learning?
Chatbot: Machine learning adalah cabang AI yang membuat mesin belajar dari data.
Anda: supervised sama unsupervised, beda nya tuh apa?
Chatbot: Maaf, saya tidak dapat membantu dengan pertanyaan itu.
Anda: oke, kalau presiden indonesia?
Chatbot: Maaf, saya tidak dapat membantu dengan pertanyaan itu.
Anda: exit
Chatbot: Terima kasih! Sampai jumpa.
